In [1]:
import pandas as pd
import json
import textwrap

In [2]:
with open("../server/ownership-inventory.log", "r") as f:
    logs = []
    for l in f.readlines():
        data = json.loads(l)
        demo = data.pop('demo', None)
        logs.append({
            **demo,
            **data
        })
    logs = pd.DataFrame(logs)

resp = logs.groupby('id').apply(lambda df: df.sort_values('end').iloc[-1])
resp['duration'] = (resp.end - resp.start) / (1000 * 60)

ans = []
for _, row in resp.iterrows():
    row = row.to_dict()
    for a in row.pop('answers', None):
        tasks = a.pop('answer')
        ans.append({
            **row,
            **a,
            **tasks
        })
ans = pd.DataFrame(ans)
ans = ans.groupby(['id', 'question']).apply(lambda df: df.sort_values('end').iloc[0]).reset_index(drop=True)
# ans

In [3]:
resp.sort_values('start')[['name', 'email', 'duration', 'yearsCpp']].reset_index()

,id,name,email,duration,yearsCpp
0,ae6881fa-41bc-4814-9dbe-cf80ac5733f6,Mehmet Akifhan ILGAZ,akifhanilgazz@gmail.com,27.739333,0
1,2df560f9-2ce0-468d-a4f2-b4a652a4386d,Vasiliy Kurdin,vasiliy.kurdin@gmail.com,60.717933,1
2,145b64df-29e9-40ea-acdf-365ac5beb1df,Sergei,selfuryon@gmail.com,40.423033,0
3,636e4fb7-aea7-43bf-8f2c-2782f08ae8b1,Oleg Nizhnikov,odomontois@gmail.com,37.913500,0
4,07c0a24a-3282-413b-9226-6423545caeb9,Sinan Yurteri,sinanyurteri42@gmail.com,18.317417,2
5,c7d25b04-e019-4482-acf4-56cf9bf5440b,Chester,wyke.chester@gmail.com,45.583650,0
6,5bd78036-8736-452e-92b6-e9e6f75902ea,Ali Kemal Özden,twitteralikemal@gmail.com,31.601900,1
7,0091a1e6-81c7-412b-919c-42ce44a4b0c5,Ömer Faruk,zozaktestnet@gmail.com,20.806067,0
8,4ef8f0d7-1e65-4b71-8f9e-152b35d43433,FAZEEL USMANI,fazeelusmani18@gmail.com,1237.645183,10
9,b634e101-8938-46ff-b99d-93d7cfe2db6e,Ahmet Hilmi Özdemir,enayiasd@gmail.com,20.694833,0


In [4]:
ans.groupby('question').size()

question
add_displayable     6
apply_curve         6
concat_all          6
find_nth            6
get_or_default     10
make_separator      5
remove_zeros        9
reverse             6
dtype: int64

In [5]:
for q in ans.question.unique():
    to_delete = ['id', 'start', 'end', 'commitHash', 'duration', 'question']
    ans[ans.question == q].drop(to_delete, axis=1).to_csv(f'data/{q}.csv')

In [7]:
# for q in ans.question.unique():
#     df = ans[ans.question == q]
#     print('= QUESTION:', q)
    
#     def print_col(k, k_nice):
#         print('==', k_nice)
#         for a in df[k].tolist():
#             print(textwrap.indent(a, '  '))
#             print('  ----')
#         print()
#     print_col('errorExplanation', 'Error explanation:')
#     print_col('messageInterpretation', 'Message interpretation:')
#     print_col('safetyViolation', 'Safety violation:')
#     print_col('functionFix', 'Function fix:')

    
    
#     print('='*30 + '\n')

In [41]:
import gspread
import gspread_formatting as fmt

gc = gspread.service_account()

sh = gc.open("Ownership inventory data")
all_ws = sh.worksheets()

def find_ws(title):
    for ws in all_ws:
        if ws.title == title:
            return ws

In [48]:
def ltr(i):
    return chr(65 + i)
assert(ltr(0) == 'A')

In [70]:
for q in ans.question.unique():
    to_delete = ['id', 'start', 'end', 'commitHash', 'duration', 'question']
    fields = ans[ans.question == q].drop(to_delete, axis=1)    
    cols = fields.columns
    data_i = cols.tolist().index('errorExplanation')

    ws = find_ws(q)
    if not ws:
        ws = sh.add_worksheet(title=q, rows=len(fields)+1, cols=len(cols)+3)
        
    header_fmt = fmt.cellFormat(textFormat=fmt.textFormat(bold=True))
    text_fmt = fmt.cellFormat(wrapStrategy='WRAP', verticalAlignment="TOP")
    code_fmt = fmt.cellFormat(textFormat=fmt.textFormat(fontFamily="Source Code Pro"), verticalAlignment="TOP")
    
    formats = [text_fmt, text_fmt, code_fmt, code_fmt]    
    fmt.format_cell_ranges(ws, [("A1:I1", header_fmt)] + list(zip(map(lambda i: ltr(data_i + i), range(4)), formats)))
    meta_width = 50
    widths = [200, 200, 400, 400]
    fmt.set_column_widths(ws, [(ltr(i), meta_width) for i in range(data_i)] + list(zip(map(lambda i: ltr(data_i + i), range(4)), widths)))
    
    ws.update([cols.tolist()] + fields.values.tolist())